In [22]:
import numpy as np
from scipy.linalg import lu
import random
import math

random.seed(0)

e = None

class SymmetricMatrix:
    def __init__(self, size=None, from_matrix=None):
        
        if from_matrix is not None:
            self.real_size = len(from_matrix)
            self.container = np.zeros((self.real_size * (self.real_size + 1)) // 2)
            pos = 0
            
            for i in range(self.real_size):
                for j in range(i + 1):
                    self.container[pos] = from_matrix[i,j]
                    pos += 1
                    
        else:
            self.real_size = size
            self.container = np.zeros((size * (size + 1)) // 2)
    
    def __len__(self):
        return self.real_size
    
    def get_real_coord(self, coords):
        r, c = coords
        
        if coords[1] > coords[0]:
            aux = r
            r = c
            c = aux
        
        return ((r * (r + 1)) // 2) + c
    
    def __setitem__(self, coords, value):
        real_coord = self.get_real_coord(coords)
        self.container[real_coord] = value
        
    def __getitem__(self, coords):
        real_coord = self.get_real_coord(coords)
        return self.container[real_coord]
    
    def __str__(self):
        s = ""
        to_show = 1
        pos = 0
        while pos < len(self.container):
            for i in range(to_show):
                s += str(self.container[pos]) + ","
                pos += 1
            s += "\n"
            to_show += 1
        return s
    
    def __repr__(self):
        s = ""
        to_show = 1
        pos = 0
        while pos < len(self.container):
            for i in range(to_show):
                s += str(self.container[pos]) + " "
                pos += 1
            s += "\n"
            to_show += 1
        return s

class LinearSystemReader:
    def read_linear_system_from_file(path):
        with open(path, 'r') as f:
            l = [[float(n) for n in line.split(',')] for line in f]
            A = [l[r][:-1] for r in range(len(l))]
            b = [l[r][-1] for r in range(len(l))]
            
            return (np.array(A), np.array(b))

    def read_linear_system_from_keyboard():
        r = int(input("Enter the number of rows:")) 
        c = int(input("Enter the number of columns:")) 
        l = []
        
        if r != c-1:
            print("Invalid dimensions")
            return None
        else:
            for i in range(r):
                l.append(list(map(float, input().rstrip().split())))

            A = [[l[i][j] for j in range(c - 1)] for i in range(r)]
            b = [l[i][c - 1] for i in range(r)]
            
            return (np.array(A), np.array(b))

    def write_linear_system_to_file(path, A, b):
        with open(path, 'w') as f:
            for r in range(len(A)):
                for c in range(len(A[0]) + 1):
                    if c == len(A[0]):
                        f.write(str(b[r]))
                    else:
                        f.write(str(A[r][c]) + ",")
                f.write("\n")
                
    def write_linear_system_to_screen(A, b):
        for r in range(len(A)):
            l = ""
            for c in range(len(A[0]) + 1):
                if c == len(A[0]):
                    l += str(b[r])
                else:
                    l += str(A[r][c]) + ","
            print(l)

class SymmetricMatrixGenerator:
    def random_matrix(size):
        A = np.random.randint(-9999, 9999, (size,size))
        symm = (A + A.T) / 2
        
        return symm
    
    def random_positive_matrix(size):
        A = np.random.rand(size,size)
        symm = (A + A.T) / 2
        
        return symm + (size * np.eye(size))

class CholeskyDecomposer:
    def decompose(A, d):
        global e
        n = len(A)
        
        for i in range(n):
            for p in range(i + 1):
                prod_sum = sum(A[i,j] * A[p,j] for j in range(p))
                
                if i == p:
                    A[i,p] = math.sqrt(d[i] - prod_sum)
                elif math.fabs(A[p,p] > e):
                    A[i,p] = (A[i,p] - prod_sum) / A[p,p]
    
    def decompose_restricted(A, d):
        global e
        n = len(A)
        L = SymmetricMatrix(size=n)
        
        for i in range(n):
            for p in range(i + 1):
                prod_sum = sum(L[i,j] * L[p,j] for j in range(p))
                
                if i == p:
                    L[i,p] = math.sqrt(d[i] - prod_sum)
                elif math.fabs(L[p,p] > e):
                    L[i,p] = (A[p,i] - prod_sum) / L[p,p]
        
        return L
    
def det_triangular_matrix(A):
    det = 1
    for i in range(len(A)):
        det *= A[i,i]
        
    return det

def solve_system_substitution_method(L, b):
    n = len(b)
    nl = len(L)
    x = np.zeros(n)
    
    for i in range(n):
        prod_sum = sum(L[i,j] * x[j] for j in range(i))
        
        if math.fabs(L[i,i] > e):
            x[i] = (b[i] - prod_sum) / L[i,i]
        
    return x

def solve_system_inverse_substitution_method(L, b):
    n = len(b)
    nl = len(L)
    x = np.zeros(n)
    
    for i in range(n - 1, -1, -1):
        prod_sum = sum(L[i,j] * x[j] for j in range(i + 1, n))
        
        if math.fabs(L[i,i] > e):
            x[i] = (b[i] - prod_sum) / L[i,i]
        
    return x

def lu_decomposition(A):
    p, l, u = lu(A)
    
    return (l, u)

def matrix_inverse(A):
    n = len(A)
    I = np.eye(n)
    A_inverse = np.zeros((n,n))
    
    for i in range(0, n):
        y_star = solve_system_substitution_method(A, I[i])
        x_star = solve_system_inverse_substitution_method(A.T, y_star)
        A_inverse[:,i] = x_star
        
    return A_inverse

def input_error():
    global e
    m = int(input("Enter m:"))
    e = 10 ** (-m)
    print("Error: %f"%e)
    
def extract_diagonal(A):
    d = []
    for i in range(len(A)):
        d.append(A[i,i])
        
    return d

def extract_testing_data(A, d):
    n = len(A)
    L = np.zeros((n,n))
    A_init = np.zeros((n,n))
    
    for i in range(0, n):
        for j in range(0, n):
            if j < i:
                L[i,j] = A[i,j]
            if j > i:
                L[i,j] = 0
                A_init[i,j] = A[i,j]
                A_init[j,i] = A[i,j]
            if i == j:
                L[i,j] = A[i,j]
                A_init[i,j] = d[i]
            
    return A_init, L, L.T
    

In [23]:
input_error()

Enter m:5
Error: 0.000010


In [24]:
A, b = LinearSystemReader.read_linear_system_from_file("./data/linear_system_example.csv")
A, b

(array([[4., 2., 4.],
        [2., 2., 2.],
        [4., 2., 5.]]),
 array([12.,  6., 13.]))

In [25]:
d = extract_diagonal(A)
d

[4.0, 2.0, 5.0]

In [26]:
CholeskyDecomposer.decompose(A, d)
A

array([[2., 2., 4.],
       [1., 1., 2.],
       [2., 0., 1.]])

In [27]:
A_init, L, L_T = extract_testing_data(A, d)
A_init, L, L_T

(array([[4., 2., 4.],
        [2., 2., 2.],
        [4., 2., 5.]]),
 array([[2., 0., 0.],
        [1., 1., 0.],
        [2., 0., 1.]]),
 array([[2., 1., 2.],
        [0., 1., 0.],
        [0., 0., 1.]]))

In [28]:
np.linalg.norm(A_init - L @ L_T)

0.0

In [29]:
det_A = det_triangular_matrix(A) * det_triangular_matrix(A)
det_A

4.0

In [30]:
np.linalg.norm(np.linalg.det(A_init) - det_A)

0.0

In [31]:
L

array([[2., 0., 0.],
       [1., 1., 0.],
       [2., 0., 1.]])

In [32]:
solve_system_substitution_method(A, b)

array([6., 0., 1.])

In [33]:
y_star = solve_system_substitution_method(A, b)
x_star = solve_system_inverse_substitution_method(A.T, y_star)
x_star

array([2., 0., 1.])

In [34]:
np.linalg.norm(A_init @ x_star - b)

0.0

In [35]:
L_lu, U_lu = lu_decomposition(A_init)
L_lu, U_lu

(array([[1. , 0. , 0. ],
        [0.5, 1. , 0. ],
        [1. , 0. , 1. ]]),
 array([[4., 2., 4.],
        [0., 1., 0.],
        [0., 0., 1.]]))

In [36]:
y_star_lu = solve_system_substitution_method(L_lu, b)
x_star_lu = solve_system_inverse_substitution_method(U_lu, y_star_lu)
x_star_lu

array([2., 0., 1.])

In [37]:
np.linalg.norm(A_init @ x_star_lu - b)

0.0

In [38]:
A_inverse = matrix_inverse(A)
A_inverse

array([[ 1.5, -0.5, -1. ],
       [-0.5,  1. ,  0. ],
       [-1. ,  0. ,  1. ]])

In [39]:
np.linalg.norm(A_init @ A_inverse - np.eye(len(A_init), len(A_init)))

0.0

In [40]:
np.linalg.norm(A_inverse - np.linalg.inv(A_init))

0.0

In [41]:
L = CholeskyDecomposer.decompose_restricted(A, d)
L

2.0 
1.0 1.0 
2.0 0.0 1.0 

In [42]:
solve_system_substitution_method(L, b)

array([6., 0., 1.])